THIS NOTEBOOK CONTAINS THE PROJECT RAW CODE FOR IMAGE ROATION DETECTION USING IN PRETRAININED MODEL (RESNET)

OBJETIVE HERE IS TO GET THE ANGLE OF ROTATION FOR AN IMAGE:


Step1. We are generating images with diffrent ratation (0-40) and placing them on raw image at with randaom offset/placement.

Step2: We are creating 2 datagenerator for testing and validation

Step3: Creating Resnet, training it


In [ ]:
# import cv2
# import matplotlib.pyplot as plt
from PIL import Image 
import numpy as np
import os
from tqdm import tqdm

import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
import cv2
import os 
import numpy as np
import matplotlib.image as mpimg
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [ ]:
file_name = "local.jpeg"

src = cv2.imread(file_name, 1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
# cv2.imwrite("test.png", dst)
cnts = cv2.findContours(threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
cnt = sorted(cnts, key=cv2.contourArea)[-1]

x,y,w,h = cv2.boundingRect(cnt)
dst = src[y:y+h, x:x+w]

In [ ]:
_,blank_image = cv2.threshold(gray, 255, 255,cv2.THRESH_BINARY_INV)

In [ ]:
img = Image.fromarray(dst)

In [ ]:
## Here we are generating the train data of images
try:
  shutil.rmtree('train_rotation')
except:
  pass
os.mkdir('train_rotation')


for i in tqdm(range(0,10)):
    img = Image.fromarray(dst)
    img = img.rotate(i,expand = 1,fillcolor=255)
    img_array = np.array(img)
    try:
        os.mkdir(f'train_rotation/{i}')
    except:
        pass
    
    
    for j in range(3000):
        l_img = 0
        s_img = 0
        s_img = img_array[:,:,0].copy()
        l_img = blank_image.copy()
        x_offset   = np.random.randint(1,100)
        y_offset   = np.random.randint(1,100)
        l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img
        im = Image.fromarray(l_img)
        im.save(f"train_rotation/{i}/{j}.png")

In [ ]:
## Here we are generating the test data of images
try:
  shutil.rmtree('test_rotation')
except:
  pass
os.mkdir('test_rotation')



for i in tqdm(range(0,10)):
    img = Image.fromarray(dst)
    img = img.rotate(i,expand = 1,fillcolor=255)
    img_array = np.array(img)
    try:
        os.mkdir(f'test_rotation/{i}')
    except:
        pass
    
    
    for j in range(30):
        l_img = 0
        s_img = 0
        s_img = img_array[:,:,0].copy()
        l_img = blank_image.copy()
        x_offset   = np.random.randint(1,100)
        y_offset   = np.random.randint(1,100)
        l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img
        im = Image.fromarray(l_img)
        im.save(f"test_rotation/{i}/{j}.png")

In [ ]:
image_hight,image_width = 224,224
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   validation_split=.4)

In [ ]:
train_generator =train_datagen.flow_from_directory('train_rotation',
                                                   target_size =(image_hight,image_width),
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   subset='training')

valid_generator =train_datagen.flow_from_directory('test_rotation',
                                                   target_size =(image_hight,image_width),
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   subset= "validation")

In [ ]:
from keras.layers.attention.multi_head_attention import activation
from keras.layers import GlobalAveragePooling2D
base_model = ResNet50(include_top=False,weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x= Dense(1024,activation='relu')(x)
predictions =Dense(train_generator.num_classes,activation='softmax')(x)
model = Model(inputs= base_model.input,outputs=predictions)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_generator,validation_data=valid_generator,epochs=10)

563/563 [==============================] - 185s 327ms/step - loss: 0.0622 - accuracy: 0.9854 - val_loss: 0.6028 - val_accuracy: 0.8250
<keras.callbacks.History at 0x7f87ea5bf9d0>

In [ ]:
model.save('final_model.hdf5')